# 函数
常见的规则：如果改变了入参，则返回None，反之，返回有意义的数字，如`list.sort`和`sorted`

In [3]:
a = [1, 3, 2, 5, 7, 6]
print(sorted(a))
print(a)
print(a.sort())
print(a)

[1, 2, 3, 5, 6, 7]
[1, 3, 2, 5, 7, 6]
None
[1, 2, 3, 5, 6, 7]


## 可变长参数 *args